In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import pandas as pd
import numpy as np
import os
import zipfile
from zipfile import ZipFile

In [ ]:
#ncm_exportacao
for ano in [*range(1997, 2022)]:
  for mes in [*range(1, 13)]:
    directory = '/content/drive/MyDrive/br_me_comex_stat/output/ncm_exportacao/ano={}/mes={}'.format(ano, mes)
    if not os.path.exists(directory):
      os.makedirs(directory)

#ncm_importacao
for ano in [*range(1997, 2022)]:
  for mes in [*range(1, 13)]:
    directory = '/content/drive/MyDrive/br_me_comex_stat/output/ncm_importacao/ano={}/mes={}'.format(ano, mes)
    if not os.path.exists(directory):
      os.makedirs(directory)

#municipio_exportacao
for ano in [*range(1997, 2022)]:
  for mes in [*range(1, 13)]:
    directory = '/content/drive/MyDrive/br_me_comex_stat/output/municipio_exportacao/ano={}/mes={}'.format(ano, mes)
    if not os.path.exists(directory):
      os.makedirs(directory)
      
#municipio_importacao
for ano in [*range(1997, 2022)]:
  for mes in [*range(1, 13)]:
    directory = '/content/drive/MyDrive/br_me_comex_stat/output/municipio_importacao/ano={}/mes={}'.format(ano, mes)
    if not os.path.exists(directory):
      os.makedirs(directory)

In [ ]:
#ncm_exportacao
!wget --no-check-certificate https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/EXP_COMPLETA.zip

In [ ]:
ncm_exp = os.path.join('/content/EXP_COMPLETA.zip')
with ZipFile(ncm_exp) as z:
  with z.open(f'EXP_COMPLETA.csv') as f:
    ncm_expo = pd.read_csv(f,sep=';')

rename_ncm = {'CO_ANO':'ano', 'CO_MES':'mes', 'CO_NCM':'id_ncm', 'CO_UNID':'id_unidade', 'CO_PAIS':'id_pais', 'SG_UF_NCM':'sigla_uf_ncm', 'CO_VIA':'id_via', 
              'CO_URF':'id_urf', 'QT_ESTAT':'quantidade_estatistica', 'KG_LIQUIDO':'peso_liquido_kg', 'VL_FOB':'valor_fob_dolar'}
ncm_expo.rename(columns=rename_ncm, inplace=True)
for ano in [*range(1997, 2022)]:
  for mes in [*range(1, 13)]:
    print("Particionando mês {} de {}".format(str(mes), str(ano)))
    ncm_ano = ncm_expo[ncm_expo['ano'] == ano]
    ncm_ano_mes = ncm_ano[ncm_ano['mes'] == mes]
    ncm_ano_mes.drop(['ano', 'mes'], axis=1, inplace=True)
    exec("ncm_ano_mes.to_csv('/content/drive/MyDrive/br_me_comex_stat/output/ncm_exportacao/ano={}/mes={}/ncm_exportacao.csv', index=False, encoding='utf-8', na_rep='')".format(ano, mes))
del ncm_expo, ncm_ano, ncm_ano_mes

In [ ]:
#ncm_importacao
!wget --no-check-certificate https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/IMP_COMPLETA.zip

In [ ]:
ncm_imp = os.path.join('/content/IMP_COMPLETA.zip')
with ZipFile(ncm_imp) as z:
  with z.open(f'IMP_COMPLETA.csv') as f:
    ncm_impo = pd.read_csv(f,sep=';')

ncm_impo.rename(columns=rename_ncm, inplace=True)

for ano in [*range(1997, 2022)]:
  for mes in [*range(1, 13)]:
    print("Particionando mês {} de {}".format(str(mes), str(ano)))
    ncm_ano = ncm_impo[ncm_impo['ano'] == ano]
    ncm_ano_mes = ncm_ano[ncm_ano['mes'] == mes]
    ncm_ano_mes.drop(['ano', 'mes'], axis=1, inplace=True)
    exec("ncm_ano_mes.to_csv('/content/drive/MyDrive/br_me_comex_stat/output/ncm_importacao/ano={}/mes={}/ncm_importacao.csv', index=False, encoding='utf-8', na_rep='')".format(ano, mes))
del ncm_impo, ncm_ano, ncm_ano_mes

In [ ]:
#municipio_exportacao
!wget --no-check-certificate https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/EXP_COMPLETA_MUN.zip

In [2]:
id_mun = pd.read_csv('/content/drive/MyDrive/br_ms_sinasc/id_mun.csv')
id_mun.drop(['regiao', 'id_municipio_6'], axis=1, inplace=True)

In [ ]:
municipio_exp = os.path.join('/content/EXP_COMPLETA_MUN.zip')
with ZipFile(municipio_exp) as z:
  with z.open(f'EXP_COMPLETA_MUN.csv') as f:
    mun_exp = pd.read_csv(f,sep=';')

rename_mun = {'CO_ANO':'ano', 'CO_MES':'mes', 'SH4':'id_sh4', 'CO_PAIS':'id_pais', 'SG_UF_MUN':'sigla_uf', 'CO_MUN':'id_municipio', 
              'KG_LIQUIDO':'peso_liquido_kg', 'VL_FOB':'valor_fob_dolar'}
mun_exp.rename(columns=rename_mun, inplace=True)

condicao = [
            ((mun_imp['sigla_uf'] == "SP") & (mun_imp['id_municipio'] < 3500000)), 
            ((mun_imp['sigla_uf'] == "MS") & (mun_imp['id_municipio'] > 5000000)),
            ((mun_imp['sigla_uf'] == "GO") & (mun_imp['id_municipio'] > 5200000)),
            ((mun_imp['sigla_uf'] == "DF") & (mun_imp['id_municipio'] > 5300000))
]
valores = [
           mun_exp['id_municipio'] + 100000,
           mun_exp['id_municipio'] - 200000,
           mun_exp['id_municipio'] - 100000,
           mun_exp['id_municipio'] - 100000
]
mun_exp['id_municipio'] = np.select(condicao, valores, default=mun_exp['id_municipio'])

for ano in [*range(1997, 2022)]:
  for mes in [*range(1, 13)]:
    print("Particionando mês {} de {}".format(str(mes), str(ano)))
    mun_ano = mun_exp[mun_exp['ano'] == ano]
    mun_ano_mes = mun_ano[mun_ano['mes'] == mes]
    mun_ano_mes.drop(['ano', 'mes'], axis=1, inplace=True)
    exec("mun_ano_mes.to_csv('/content/drive/MyDrive/br_me_comex_stat/output/municipio_exportacao/ano={}/mes={}/municipio_exportacao.csv', index=False, encoding='utf-8', na_rep='')".format(ano, mes))
del mun_exp, mun_ano, mun_ano_mes

In [ ]:
#municipio_importacao
!wget --no-check-certificate https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/IMP_COMPLETA_MUN.zip

In [ ]:
municipio_imp = os.path.join('/content/IMP_COMPLETA_MUN.zip')
with ZipFile(municipio_imp) as z:
  with z.open(f'IMP_COMPLETA_MUN.csv') as f:
    mun_imp = pd.read_csv(f,sep=';')

rename_mun = {'CO_ANO':'ano', 'CO_MES':'mes', 'SH4':'id_sh4', 'CO_PAIS':'id_pais', 'SG_UF_MUN':'sigla_uf', 'CO_MUN':'id_municipio', 
              'KG_LIQUIDO':'peso_liquido_kg', 'VL_FOB':'valor_fob_dolar'}
mun_imp.rename(columns=rename_mun, inplace=True)

condicao = [
            ((mun_imp['sigla_uf'] == "SP") & (mun_imp['id_municipio'] < 3500000)), 
            ((mun_imp['sigla_uf'] == "MS") & (mun_imp['id_municipio'] > 5000000)),
            ((mun_imp['sigla_uf'] == "GO") & (mun_imp['id_municipio'] > 5200000)),
            ((mun_imp['sigla_uf'] == "DF") & (mun_imp['id_municipio'] > 5300000))
]
valores = [
           mun_imp['id_municipio'] + 100000,
           mun_imp['id_municipio'] - 200000,
           mun_imp['id_municipio'] - 100000,
           mun_imp['id_municipio'] - 100000
]
mun_imp['id_municipio'] = np.select(condicao, valores, default=mun_imp['id_municipio'])

for ano in [*range(1997, 2022)]:
  for mes in [*range(1, 13)]:
    print("Particionando mês {} de {}".format(str(mes), str(ano)))
    mun_ano = mun_imp[mun_imp['ano'] == ano]
    mun_ano_mes = mun_ano[mun_ano['mes'] == mes]
    mun_ano_mes.drop(['ano', 'mes'], axis=1, inplace=True)
    exec("mun_ano_mes.to_csv('/content/drive/MyDrive/br_me_comex_stat/output/municipio_importacao/ano={}/mes={}/municipio_importacao.csv', index=False, encoding='utf-8', na_rep='')".format(ano, mes))
del mun_imp, mun_ano, mun_ano_mes